# Get a description of a word from DuckDuckGo API

[Cohttp](https://github.com/mirage/ocaml-cohttp) ia a lightweight HTTP client/server library. Its interface is simple and easy to use. This library is useful for web scraping and requesting to API servers. In this example, we obtain a description of a given word from the DuckDuckGo API.

In [1]:
#require "cohttp,cohttp-lwt-unix,ppx_yojson_conv" ;;

In [2]:
open Lwt.Infix ;;

In [3]:
type t = {
    definition : string [@key "Definition"];
    abstract : string [@key "Abstract"];
  } [@@deriving yojson] [@@yojson.allow_extra_fields]

type t = { definition : string; abstract : string; }
val t_of_yojson : Ppx_yojson_conv_lib.Yojson.Safe.t -> t = <fun>
val yojson_of_t : t -> Ppx_yojson_conv_lib.Yojson.Safe.t = <fun>


In [4]:
let query = "OCaml" (* Search query *)
let body =
  Lwt_main.run begin
    let base_uri = Uri.of_string "http://api.duckduckgo.com/?format=json" in
    let uri = Uri.add_query_param base_uri ("q", [query]) in
    Cohttp_lwt_unix.Client.get uri >>= fun (resp, body) -> (* GET contents from a given uri *)
    assert (Cohttp.Response.status resp = `OK) ; (* Check HTTP response code *)
    Cohttp_lwt.Body.to_string body >|= fun body -> (* Receive contents *)
    Yojson.Safe.from_string body
    |> [%of_yojson: t]
    |> function
    | { definition = ""; abstract = ""; } -> None
    | { definition = ""; abstract; } -> Some abstract
    | { definition; _ } -> Some definition
  end

val query : string = "OCaml"


val body : string option =
  Some
   "OCaml is a general-purpose, multi-paradigm programming language which extends the Caml dialect of ML with object-oriented features. OCaml was created in 1996 by Xavier Leroy, Jérôme Vouillon, Damien Doligez, Didier Rémy, Ascánder Suárez, and others. The OCaml toolchain includes an interactive"... (* string length 994; truncated *)
